In [2]:
from pyculib.sparse import Sparse
import pyculib.sparse as sp
import numpy as np
import scipy
import time
import pandas as pd
import scipy.io as sio

/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [64]:
# Construct the table of reference to map N/density to the file names.
table = pd.DataFrame(index=['0.0001', '0.001', '0.01', '0.1'], columns=['4000', '8000', '40000', '80000', '160000'])
results = pd.DataFrame(index=['0.0001', '0.001', '0.01', '0.1'], columns=['4000', '8000', '40000', '80000', '160000'])
table['4000']['0.0001'] = 'bcsstm24.mat'
table['4000']['0.001'] = 'c-24.mat'
table['4000']['0.01'] = 'crystk01.mat'
table['4000']['0.1'] = 'heart1.mat'
table['8000']['0.0001'] = 'bcsstm38.mat'
table['8000']['0.001'] = 'c-39.mat'
table['8000']['0.01'] = 'msc10848.mat'
table['8000']['0.1'] = 'human_gene2.mat' # previously 'TSC_OPF_1047.mat' 
table['40000']['0.0001'] = 'cond-mat-2005.mat'
table['40000']['0.001'] = 'bbmat.mat'
table['40000']['0.01'] = 'TSOPF_RS_b2383_c1.mat'
table['40000']['0.1'] = '' # Does not exist.
table['80000']['0.0001'] = 'net4-1.mat'
table['80000']['0.001'] = 'consph.mat'
table['80000']['0.01'] = '' # Does not exist
table['80000']['0.1'] = '' # Does not exist
table['160000']['0.0001'] = 'para-4.mat'
table['160000']['0.001'] = 'pkustk14.mat'
table['160000']['0.01'] = '' # Does not exist
table['160000']['0.1'] = '' # Does not exist
table

,4000,8000,40000,80000,160000
0.0001,bcsstm24.mat,bcsstm38.mat,cond-mat-2005.mat,net4-1.mat,para-4.mat
0.001,c-24.mat,c-39.mat,bbmat.mat,consph.mat,pkustk14.mat
0.01,crystk01.mat,msc10848.mat,TSOPF_RS_b2383_c1.mat,,
0.1,heart1.mat,human_gene2.mat,,,


In [65]:
# Handle all the edge cases for all files
def load_matrix(matrix_name):
    data = sio.loadmat(matrix_name)
    P = data['Problem']
    zeros = {'net4-1.mat'}
    twos = {'consph.mat','human_gene2.mat','TSOPF_RS_b2383_c1.mat','cond-mat-2005.mat', 'para-4.mat'}
    fours = {'c-24.mat','c-39.mat'}
    if matrix_name in zeros:
        x = P[0][0][0]
    elif matrix_name in twos:
        x = P[0][0][2]
    elif matrix_name in fours:
        x = P[0][0][4]
    else:
        x = P[0][0][1]
    return x

def multiply_and_time(matrix_name, N, p, results):
    if not matrix_name:
        return
    s = sp.Sparse()
    a = load_matrix(matrix_name)
    b = a[:]
    ap, bp = sp.csr_matrix(a), sp.csr_matrix(b)
    start = time.time()
    sp.Sparse.csrgemm_ez(s, ap, bp)
    end = time.time()
    print("\tTime for {0} = {1}".format(matrix_name, end-start))
    results[N][p] = end-start
    return


def multiply_all(table, results, reverse=False, skip_computed=False):
    N_list = ['4000', '8000', '40000', '80000', '160000']
    p_list = ['0.0001', '0.001', '0.01', '0.1']
    if reverse:
        N_list = list(reversed(N_list))
        p_list = list(reversed(p_list))
    for N in N_list:
        for p in p_list:
            print("For N={0}, p={1}".format(N,p))
            if not results.isna()[N][p] and skip_computed:
                print("\t Skipping, already computed.")
            else:
                try:
                    multiply_and_time(table[N][p], N, p, results)
                except:
                    print("\tException caught: CUDA_ERROR_OUT_OF_MEMORY Error!")
            

In [68]:
multiply_all(table, results, reverse=False, skip_computed=True)
print("Done!")

For N=4000, p=0.0001
	 Skipping, already computed.
For N=4000, p=0.001
	 Skipping, already computed.
For N=4000, p=0.01
	 Skipping, already computed.
For N=4000, p=0.1
	 Skipping, already computed.
For N=8000, p=0.0001
	 Skipping, already computed.
For N=8000, p=0.001
	 Skipping, already computed.
For N=8000, p=0.01
	 Skipping, already computed.
For N=8000, p=0.1
	 Skipping, already computed.
For N=40000, p=0.0001
	 Skipping, already computed.
For N=40000, p=0.001
	 Skipping, already computed.
For N=40000, p=0.01
	 Skipping, already computed.
For N=40000, p=0.1
For N=80000, p=0.0001
	Exception caught: CUDA_ERROR_OUT_OF_MEMORY Error!
For N=80000, p=0.001
	 Skipping, already computed.
For N=80000, p=0.01
For N=80000, p=0.1
For N=160000, p=0.0001
	 Skipping, already computed.
For N=160000, p=0.001
	 Skipping, already computed.
For N=160000, p=0.01
For N=160000, p=0.1
Done!


In [69]:
results

,4000,8000,40000,80000,160000
0.0001,0.00119758,0.00169945,0.297454,NaN,2.43103
0.001,0.0341461,0.097985,0.169567,0.820481,3.43052
0.01,0.0527699,0.411427,0.446058,NaN,NaN
0.1,1.15618,69.3924,NaN,NaN,NaN


## CSR x Dense Matrix

In [55]:
results_2 = pd.DataFrame(index=['0.0001', '0.001', '0.01', '0.1'], columns=['4000', '8000', '40000', '80000', '160000'])

In [58]:
def multiply_all(table, results, reverse=False, skip_computed=False):
    N_list = ['4000', '8000']
    p_list = ['0.0001', '0.001', '0.01', '0.1']
    if reverse:
        N_list = list(reversed(N_list))
        p_list = list(reversed(p_list))
    for N in N_list:
        for p in p_list:
            print("For N={0}, p={1}".format(N,p))
            if not results.isna()[N][p] and skip_computed:
                print("\t Skipping, already computed.")
            else:
                try:
                    multiply_and_time(table[N][p], N, p, results)
                except:
                    print("\tException caught: CUDA_ERROR_OUT_OF_MEMORY Error!")

def multiply_and_time(matrix_name, N, p, results):
    if not matrix_name:
        return
    s = sp.Sparse()
    a = load_matrix(matrix_name)
    N_x = a.shape[0]
    b = scipy.sparse.random(N_x,N_x, 0.1, "csr")
    ap, bp = sp.csr_matrix(a), sp.csr_matrix(b)
    start = time.time()
    sp.Sparse.csrgemm_ez(s, ap, bp)
    end = time.time()
    print("\tTime for {0} = {1}".format(matrix_name, end-start))
    results[N][p] = end-start
    return



multiply_all(table, results_2, skip_computed=True)
print("Done!")

For N=4000, p=0.0001
	 Skipping, already computed.
For N=4000, p=0.001
	 Skipping, already computed.
For N=4000, p=0.01
	 Skipping, already computed.
For N=4000, p=0.1
	 Skipping, already computed.
For N=8000, p=0.0001
	 Skipping, already computed.
For N=8000, p=0.001
	 Skipping, already computed.
For N=8000, p=0.01
	 Skipping, already computed.
For N=8000, p=0.1
	Exception caught: CUDA_ERROR_OUT_OF_MEMORY Error!
Done!


In [57]:
results_2

,4000,8000,40000,80000,160000
0.0001,0.0328615,0.0369437,NaN,NaN,NaN
0.001,0.170958,0.287254,NaN,NaN,NaN
0.01,0.186676,1.39418,NaN,NaN,NaN
0.1,0.485632,NaN,NaN,NaN,NaN


## CSR x Dense Vector

In [61]:
results_3 = pd.DataFrame(index=['0.0001', '0.001', '0.01', '0.1'], columns=['4000', '8000', '40000', '80000', '160000'])

In [62]:
def multiply_all(table, results, reverse=False, skip_computed=False):
    N_list = ['4000', '8000', '40000', '80000', '160000']
    p_list = ['0.0001', '0.001', '0.01', '0.1']
    if reverse:
        N_list = list(reversed(N_list))
        p_list = list(reversed(p_list))
    for N in N_list:
        for p in p_list:
            print("For N={0}, p={1}".format(N,p))
            if not results.isna()[N][p] and skip_computed:
                print("\t Skipping, already computed.")
            else:
                multiply_and_time(table[N][p], N, p, results)


def multiply_and_time(matrix_name, N, p, results):
    if not matrix_name:
        return
    s = sp.Sparse()
    a = load_matrix(matrix_name)
    N_x = a.shape[0]
    b = scipy.sparse.random(N_x,1, 0.1, "csr")
    ap, bp = sp.csr_matrix(a), sp.csr_matrix(b)
    start = time.time()
    sp.Sparse.csrgemm_ez(s, ap, bp)
    end = time.time()
    print("\tTime for {0} = {1}".format(matrix_name, end-start))
    results[N][p] = end-start
    return



multiply_all(table, results_3, skip_computed=True)
print("Done!")

For N=4000, p=0.0001
	Time for bcsstm24.mat = 0.0009505748748779297
For N=4000, p=0.001
	Time for c-24.mat = 0.0010235309600830078
For N=4000, p=0.01
	Time for crystk01.mat = 0.0027534961700439453
For N=4000, p=0.1
	Time for heart1.mat = 0.006175994873046875
For N=8000, p=0.0001
	Time for bcsstm38.mat = 0.0008571147918701172
For N=8000, p=0.001
	Time for c-39.mat = 0.001371622085571289
For N=8000, p=0.01
	Time for msc10848.mat = 0.004664182662963867
For N=8000, p=0.1
	Time for human_gene2.mat = 0.05570673942565918
For N=40000, p=0.0001
	Time for cond-mat-2005.mat = 0.021091222763061523
For N=40000, p=0.001
	Time for bbmat.mat = 0.0059375762939453125
For N=40000, p=0.01
	Time for TSOPF_RS_b2383_c1.mat = 0.04684591293334961
For N=40000, p=0.1
For N=80000, p=0.0001
	Time for net4-1.mat = 0.010743379592895508
For N=80000, p=0.001
	Time for consph.mat = 0.0151519775390625
For N=80000, p=0.01
For N=80000, p=0.1
For N=160000, p=0.0001
	Time for para-4.mat = 0.02016901969909668
For N=160000, p

In [63]:
results_3

,4000,8000,40000,80000,160000
0.0001,0.000950575,0.000857115,0.0210912,0.0107434,0.020169
0.001,0.00102353,0.00137162,0.00593758,0.015152,0.0494952
0.01,0.0027535,0.00466418,0.0468459,NaN,NaN
0.1,0.00617599,0.0557067,NaN,NaN,NaN


### Ignore everything below. Used for testing & playgrounding.

In [27]:
matrix1 = 'TSOPF_RS_b2383_c1.mat'
matrix1_data = sio.loadmat(matrix1)
P = matrix1_data['Problem']
a = P[0][0][2]
b = a[:]
c = a[:]
s = sp.Sparse()
a_desc = s.matdescr(indexbase=0,matrixtype='G')
b_desc = s.matdescr(indexbase=0,matrixtype='G')
c_desc = s.matdescr(indexbase=0,matrixtype='G')

N = a.shape[0]
ap, bp, cp = sp.csr_matrix(a), sp.csr_matrix(b), sp.csr_matrix(c)
transA, transB, transC = 'N', 'N', 'N'
#sp.Sparse.csrmm2(s, transA, transB, N, N, N, len(a.data), 1.0, a_desc, ap.data, ap.indptr, ap.indices, b, N, 1.0, c, N)
#sp.Sparse.csrgemm(s, transA, transB, N, N, N, a_desc, len(a.data), ap.data, ap.indptr, ap.indices, b_desc, len(b.data), bp.data, bp.indptr, bp.indices, c_desc, cp.data, cp.indptr, cp.indices)

start = time.time()
sp.Sparse.csrgemm_ez(s, ap, bp)
end = time.time()
print("Time for {0}".format(matrix1))
print(end-start)

Time for TSOPF_RS_b2383_c1.mat
0.5271131992340088


In [11]:
pkustk14.mat

(8032, 8032)

In [28]:
matrix1 = 'pkustk14.mat'
matrix1_data = sio.loadmat(matrix1)
P = matrix1_data['Problem']
a = P[0][0][1]
b = a[:]
c = a[:]
s = sp.Sparse()
a_desc = s.matdescr(indexbase=0,matrixtype='G')
b_desc = s.matdescr(indexbase=0,matrixtype='G')
c_desc = s.matdescr(indexbase=0,matrixtype='G')

N = a.shape[0]
ap, bp, cp = sp.csr_matrix(a), sp.csr_matrix(b), sp.csr_matrix(c)
transA, transB, transC = 'N', 'N', 'N'
sp.Sparse.csrgemm(s, transA, transB, N, N, N, a_desc, len(a.data), ap.data, ap.indptr, ap.indices, b_desc, len(b.data), bp.data, bp.indptr, bp.indices, c_desc, cp.data, cp.indptr, cp.indices)
start = time.time()
sp.Sparse.csrgemm_ez(s, ap, bp)
end = time.time()
print("Time for {0}".format(matrix1))
print(end-start)

Time for pkustk14.mat
3.433053970336914


In [32]:
ap.nonzeros()

AttributeError: 'CudaCSRMatrix' object has no attribute 'nonzeros'